<a href="https://colab.research.google.com/github/RP335/microtonal_experiments/blob/main/microtonal_parrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generator for 180 midi files for the dataset. different kinds of edo classes and only triads.

In [32]:
!sudo apt-get install python@3.11.7
!python3 --version
!pip install mido
!apt-get install fluidsynth
!pip install midi2audio
!pip install music21
!pip install xenharmlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python3.11.7
E: Couldn't find any package by glob 'python3.11.7'
E: Couldn't find any package by regex 'python3.11.7'
Python 3.10.12
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.11.7; 0.1.1 Requires-Python >=3.11.7
ERROR: Could not find a version that satisfies the requirement xenharmlib (from versions: none)
ERROR: No matching distribution found for xenharmlib


installs

In [24]:
import mido
import os
from midi2audio import FluidSynth
import librosa
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from music21 import stream, note, midi, pitch
# import xenharmlib as xh


In [33]:
!rm -rf microtonal_audio_files microtonal_midi_files

In [34]:
# this requires 3.11.7 or greater which doesn't work with colab
# from xenharmlib import EDOTuning, export, UpDownNotation

# edos = [31, 19, 11]
# for edo in edos:
#     edoTuning = EDOTuning(edo)
#     n_edo = UpDownNotation(edoTuning)
#     chord = n_edo.note_scale(
#         [n_edo.note(s, 4) for s in ['C', 'E', 'G', 'Bb']]
#     )
#     filename = f'microtonal_chord_{edo}edo.wav'
#     export.audio.export_wav(filename, chord, duration = 2.0, play_as_chord= True, sample_rate=22500)


imports

In [20]:

# # Paths
# soundfont_path = '/usr/share/sounds/sf2/FluidR3_GM.sf2'
# midi_dir = 'microtonal_midi_files'
# output_dir_audio = 'microtonal_audio_files'

# os.makedirs(output_dir_audio, exist_ok=True)

# # Soundfont initialization
# fs = FluidSynth(soundfont_path)

# # Midi 2 audio
# for edo in edos:
#     for i in range(num_files_per_edo):
#         midi_file = os.path.join(midi_dir, f'{edo}edo_chord_{i+1}.mid')
#         audio_file = os.path.join(output_dir_audio, f'{edo}edo_chord_{i+1}.wav')

#         if os.path.exists(midi_file):
#             fs.midi_to_audio(midi_file, audio_file)
#         else:
#             print(f"no file exists like this: {midi_file}")

# print(f'Converted all MIDI in d {midi_dir} directory.')


Converted all MIDI in d microtonal_midi_files directory.


In [15]:

def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

audio_dir = 'microtonal_audio_files'

features = []
labels = []

# debug
print(f"Files in directory {audio_dir}: {os.listdir(audio_dir)}")

# Feature extraction
for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        # Debugging print statements
        print(f"Processing file: {file_path}")
        edo_label = file_name.split('edo')[0]
        feature = extract_features(file_path)
        features.append(feature)
        labels.append(edo_label)

if not features:
    print("No features were extracted. Check if the files are correctly named and can be read by librosa.")
else:
    df = pd.DataFrame(features)
    df['label'] = labels

    df.to_csv('microtonal_dataset.csv', index=False)

    print(f'Dataset saved with {len(features)} samples.')


Files in directory microtonal_audio_files: []
No features were extracted. Check if the files are correctly named and can be read by librosa.


In [ ]:
# Load dataset
dataset = pd.read_csv('microtonal_dataset.csv')
X = dataset.drop(columns=['label']).values
y = dataset['label'].values

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Reshape X for CNN input (single channel audio)
X = X.reshape(X.shape[0], X.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = tf.keras.models.Sequential()

# TDNN layers (1D Convolutional layers)
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

# LSTM layer
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(64))

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')
